# Projeto 3 - Ciência dos Dados

#  Projeto UFC

Nome: Victor Vergara Arcoverde de Albuquerque Cavalcanti

Nome: Edgard Neto

Nome: Gabriel Yamashita

Nome: Henrique Mualem Marti



  ___
## Objetivo:

### O objetivo desse projeto é fazer um machine learning para poder prever qual lutador ganhará uma luta do UFC baseado no seu histórico. Para isso serão usados os dados de todas as lutas do UFC (mais de 5 mil) a fim de descobrir quais os fatores dos lutadores que impactam mais no resultado das lutas. 

[Database utilizado](https://www.kaggle.com/rajeevw/ufcdata#data.csv)

 ___
## Preparando o ambiente no jupyter:

### Imports:

In [1]:
import math
import os.path
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import json
import random
import statsmodels.api as sm
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

### Trabalhando com os Excels:

In [3]:
data = pd.read_excel("data.xlsx")
#preproc = pd.read_excel("preprocessed_data.xlsx")
#fighter = pd.read_excel("raw_fighter_details.xlsx")
#total = pd.read_excel("raw_total_fight_data.xlsx")

data.head(2)

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Henry Cejudo,Marlon Moraes,Marc Goddard,2019,"Chicago, Illinois, USA",Red,True,Bantamweight,5,0,...,2,0,0,8,Orthodox,162.56,162.56,135.0,31.0,32.0
1,Valentina Shevchenko,Jessica Eye,Robert Madrigal,2019,"Chicago, Illinois, USA",Red,True,Women's Flyweight,5,0,...,0,2,0,5,Southpaw,165.10,167.64,125.0,32.0,31.0


In [4]:
data.weight_class = data.weight_class.astype('category')
data.Winner = data.Winner.astype('category')

In [5]:
data.Winner.value_counts()

Red     3470
Blue    1591
Draw      83
Name: Winner, dtype: int64

In [6]:
data_heavy = data.loc[(data.weight_class=='Heavyweight'),:]
bool_to_number = {False: 0, True: 1}
string_to_number = {'Blue': 0, 'Red': 1, 'Draw': 2}
data_heavy['title_bout'] = data_heavy['title_bout'].map(bool_to_number)
data_heavy['Winner'] = data_heavy['Winner'].map(string_to_number)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


#### Blue = 0
#### Red = 1

In [7]:
data_heavy.head(2)

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4,Tai Tuivasa,Blagoy Ivanov,Dan Miragliotta,2019,"Chicago, Illinois, USA",0,0,Heavyweight,3,0,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0
54,Walt Harris,Serghei Spivac,Jerin Valel,2019,"Ottawa, Ontario, Canada",1,0,Heavyweight,3,0,...,4,0,0,4,Southpaw,195.58,195.58,250.0,24.0,35.0


In [8]:
data_heavy_util = data_heavy.drop(['Referee','date','location'], axis=1)
#dados que não se relacionam com os lutadores ou seus resultado

In [9]:
data_heavy_util.head(2)

,R_fighter,B_fighter,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4,Tai Tuivasa,Blagoy Ivanov,0,0,Heavyweight,3,0,1,0,17.0,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0
54,Walt Harris,Serghei Spivac,1,0,Heavyweight,3,0,0,0,NaN,...,4,0,0,4,Southpaw,195.58,195.58,250.0,24.0,35.0
60,Arjan Bhullar,Juan Adams,1,0,Heavyweight,3,0,1,0,8.0,...,0,0,0,2,Orthodox,185.42,190.50,245.0,27.0,32.0
63,Greg Hardy,Dmitrii Smoliakov,1,0,Heavyweight,3,2,0,0,5.0,...,0,0,0,0,Orthodox,195.58,203.20,265.0,36.0,30.0
69,Andrei Arlovski,Augusto Sakai,0,0,Heavyweight,3,0,1,0,35.0,...,9,2,0,16,Orthodox,190.50,195.58,240.0,27.0,40.0


In [10]:
data_heavy_util.dropna(inplace=True)
data_heavy_util.head()

,R_fighter,B_fighter,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4,Tai Tuivasa,Blagoy Ivanov,0,0,Heavyweight,3,0,1,0,17.00,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0
60,Arjan Bhullar,Juan Adams,1,0,Heavyweight,3,0,1,0,8.00,...,0,0,0,2,Orthodox,185.42,190.50,245.0,27.0,32.0
63,Greg Hardy,Dmitrii Smoliakov,1,0,Heavyweight,3,2,0,0,5.00,...,0,0,0,0,Orthodox,195.58,203.20,265.0,36.0,30.0
69,Andrei Arlovski,Augusto Sakai,0,0,Heavyweight,3,0,1,0,35.00,...,9,2,0,16,Orthodox,190.50,195.58,240.0,27.0,40.0
75,Alistair Overeem,Aleksei Oleinik,1,0,Heavyweight,5,0,2,0,4.25,...,6,0,0,9,Orthodox,193.04,203.20,265.0,41.0,38.0


#### Tirando os dados categóricos:

In [11]:
categoricas = [
    'R_fighter', 
    'B_fighter', 
    'weight_class', 
    'R_Stance', 
    'B_Stance', 
]

data_heavy_cat = data_heavy_util[categoricas].astype('category')
data_heavy_num = data_heavy_util.drop(categoricas, axis=1).astype('float')

### Separando os dados em testes e treinamento

In [12]:
X = data_heavy_num.drop('Winner', axis=1)
y = data_heavy_num['Winner']

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [69]:

model = LogisticRegression(max_iter=100000,solver='lbfgs', multi_class='auto')

model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100000, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [70]:
#Fatores que tem o maior peso na decisão da vitória
j=1
lista_j=list()
#for i, f in sorted(list(zip(model.feature_importances_, X_train.columns)), reverse=True):
#    print( j,'°:',f'{f}: {i}')
#    a=str(j)+'°'
#   lista_j.append(a)
#  j+=1


#### Criando um dataframe com os dados acima:

In [79]:
'''data={'Fator':X_train.columns ,'Correlação':model.feature_importances_,}
Fator_por_corr=pd.DataFrame(data)
Fator_por_corr=Fator_por_corr.sort_values(by='Correlação', ascending=False)
Fator_por_corr['Grau de Importância']=lista_j
Fator_por_corr = Fator_por_corr.set_index('Grau de Importância')
Fator_por_corr.head()'''
print('a')

a


___
## Verificando a performance


In [72]:
X_train.columns

Index(['title_bout', 'no_of_rounds', 'B_current_lose_streak',
       'B_current_win_streak', 'B_draw', 'B_avg_BODY_att', 'B_avg_BODY_landed',
       'B_avg_CLINCH_att', 'B_avg_CLINCH_landed', 'B_avg_DISTANCE_att',
       ...
       'R_win_by_Decision_Unanimous', 'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_wins', 'R_Height_cms', 'R_Reach_cms',
       'R_Weight_lbs', 'B_age', 'R_age'],
      dtype='object', length=136)

In [73]:
model.intercept_

array([-0.0007722 ,  0.00172882, -0.00095662])

In [74]:
model.coef_

array([[ 2.87051812e-02, -2.36268342e-01,  9.32909444e-01,
         3.74494950e-01,  0.00000000e+00,  4.69438190e-01,
        -2.12861650e-01,  9.04817384e-02, -5.78530360e-02,
        -1.83338126e-02,  7.22677020e-02,  2.87281004e-01,
        -3.39642946e-01, -2.37601989e-01,  1.26252759e-01,
        -5.59401319e-01,  1.27592728e-01, -2.38619390e-01,
        -1.39386578e-01, -5.96760584e-01,  3.59428929e-01,
        -3.25228280e-01,  1.87804865e-01, -6.55473207e-02,
         3.91405441e-01,  2.10818260e-01, -2.01142552e-01,
        -6.79175828e-02,  7.21775528e-02,  5.67309760e-02,
        -2.89046803e-01,  6.47156752e-02,  1.10300198e-01,
        -8.31464280e-02,  3.75105135e-01, -1.42349737e-01,
         1.15355277e-01,  5.22781654e-01, -8.14371733e-01,
         1.02242011e-02,  7.50546871e-02, -8.31977967e-01,
         2.22345613e-01, -5.09266206e-01, -3.78316193e-01,
        -2.10686905e-01,  2.97285489e-01, -3.23911321e-01,
         5.10700839e-02,  3.89667333e-02,  8.84215301e-0

In [75]:
y_pred = model.predict(X_test)

In [76]:
print(accuracy_score(y_test, y_pred))

0.3939393939393939


In [66]:
y_pred

array([1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1.,
       1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
       0., 2., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0.,
       1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1.])

In [67]:
y_test.value_counts(True)

1.0    0.666667
0.0    0.303030
2.0    0.030303
Name: Winner, dtype: float64

In [55]:
data_heavy_num.shape

(264, 137)

### Conclusão desse teste:

Tendo um acurácia de 0.6667 não é um bom resultado, visto que o modelo praticamente sempre tem como resultado o vermelho como vencedor, e como a probabilidade do lutador vermelho ganhar é de 0.6212112 ele praticamente só acerta os que o vencedor é o vermelho e erra os que o azul é o vencedor.

Assim é possível concluir que é necessário desconsiderar algumas variáveis para tentar melhorar a acurácia.

___
## Montando a Regressão Logística


## !Ainda é necessário tirar os fatores que foram julgados ter um baixo grau de importância pelo teste anterior!

In [24]:
def preparo(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp,missing='drop')
    results = model.fit()
    return results

In [25]:
Y = data_heavy_num["Winner"]
data_heavy_sem_Winner=data_heavy_num.drop('Winner',axis=1) 
X=data_heavy_sem_Winner
#np.asarray(X)

In [26]:
result = preparo(X,Y)
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Winner   R-squared:                       0.515
Model:                            OLS   Adj. R-squared:                  0.120
Method:                 Least Squares   F-statistic:                     1.305
Date:                Thu, 14 Nov 2019   Prob (F-statistic):             0.0635
Time:                        14:18:28   Log-Likelihood:                -100.66
No. Observations:                 264   AIC:                             439.3
Df Residuals:                     145   BIC:                             864.9
Df Model:                         118                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                            4.6535      2.927      1.590      0.114      -1.131      10.438
title_bout                      -0.0643      0.192     -0.334      0.739      -0.445       0.316
no_of_rounds                     0.0958      0.062      1.533      0.127      -0.028       0.219
B_current_lose_streak           -0.2204      0.073     -3.002      0.003      -0.366      -0.075
B_current_win_streak            -0.0361      0.037     -0.983      0.327      -0.109       0.037
B_draw                        1.562e-13   4.62e-14      3.377      0.001    6.48e-14    2.48e-13
B_avg_BODY_att                  -0.0701      0.040     -1.752      0.082      -0.149       0.009
B_avg_BODY_landed                0.0788      0.049      1.623      0.107      -0.017       0.175
B_avg_CLINCH_att                 0.0259      0.025      1.025      0.307      -0.024       0.076
B_avg_CLINCH_landed             -0.0234      0.035     -0.669      0.504      -0.092       0.046
B_avg_DISTANCE_att              -0.0151      0.014     -1.059      0.291      -0.043       0.013
B_avg_DISTANCE_landed            0.0141      0.019      0.727      0.469      -0.024       0.053
B_avg_GROUND_att                -0.0245      0.023     -1.059      0.291      -0.070       0.021
B_avg_GROUND_landed              0.0242      0.032      0.764      0.446      -0.038       0.087
B_avg_HEAD_att                  -0.0283      0.022     -1.283      0.201      -0.072       0.015
B_avg_HEAD_landed                0.0352      0.028      1.266      0.208      -0.020       0.090
B_avg_KD                         0.3686      0.193      1.906      0.059      -0.014       0.751
B_avg_LEG_att                    0.0846      0.059      1.436      0.153      -0.032       0.201
B_avg_LEG_landed                -0.0990      0.071     -1.386      0.168      -0.240       0.042
B_avg_PASS                       0.1028      0.075      1.375      0.171      -0.045       0.251
B_avg_REV                        0.7377      0.295      2.497      0.014       0.154       1.322
B_avg_SIG_STR_att               -0.0137      0.023     -0.606      0.546      -0.059       0.031
B_avg_SIG_STR_landed             0.0150      0.027      0.551      0.583      -0.039       0.069
B_avg_SIG_STR_pct               -1.5196      0.575     -2.644      0.009      -2.656      -0.384
B_avg_SUB_ATT                   -0.1195      0.137     -0.871      0.385      -0.391       0.152
B_avg_TD_att                    -0.0131      0.048     -0.274      0.784      -0.107       0.081
B_avg_TD_landed                 -0.0729      0.104     -0.701      0.484      -0.278       0.133
B_avg_TD_pct                     0.3372      0.275      1.228      0.222      -0.206       0.880
B_avg_TOTAL_STR_att              0.0405      0.026      1.568      0.119    

In [27]:
'''data_heavy_blue_winners = data_heavy.loc[(data_heavy.Winner==0),:]
data_heavy_blue_winners = data_heavy_blue_winners.drop(['Winner'], axis=1)
data_heavy_red_winners = data_heavy.loc[(data_heavy.Winner==1),:]
data_heavy_red_winners = data_heavy_red_winners.drop(['Winner'], axis=1)'''

"data_heavy_blue_winners = data_heavy.loc[(data_heavy.Winner==0),:]\ndata_heavy_blue_winners = data_heavy_blue_winners.drop(['Winner'], axis=1)\ndata_heavy_red_winners = data_heavy.loc[(data_heavy.Winner==1),:]\ndata_heavy_red_winners = data_heavy_red_winners.drop(['Winner'], axis=1)"

In [28]:
'''blue_winners_correlation = data_heavy_blue_winners.corr()
blue_winners_correlation.head()'''

'blue_winners_correlation = data_heavy_blue_winners.corr()\nblue_winners_correlation.head()'

In [29]:
'''red_winners_correlation = data_heavy_red_winners.corr()
red_winners_correlation.head()'''

'red_winners_correlation = data_heavy_red_winners.corr()\nred_winners_correlation.head()'

___
## Concluindo

   ___
## Aperfeiçoamento:


   ___
## Referências

[Como usar a biblioteca Scikit-lear](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)

[Como funciona o Random Forest](https://towardsdatascience.com/understanding-random-forest-58381e0602d2)

[Referencia do Random Forest Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

[Como funciona regressão linear](https://www.saedsayad.com/logistic_regression.htm)

[Como usar a Regressão Linear](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

